<a href="https://colab.research.google.com/github/ilhamhs-blip/data_sains_project/blob/main/modul_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ucimlrepo

In [ ]:
!pip install dash

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
auto_mpg = fetch_ucirepo(id=9)
df = auto_mpg.data.original
df.info()

In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print("Missing value tiap kolom:")
print(df.isnull().sum())
median_hp = df['horsepower'].median()

df['origin'] = pd.to_numeric(df['origin'], errors='coerce')
df['origin'] = df['origin'].map({
    1: 'USA',
    2: 'Europe',
    3: 'Japan'
})

df['horsepower'] = df['horsepower'].fillna(median_hp)
print("\nSetelah imputasi:")
print(df.isnull().sum())

In [ ]:
import dash
import pandas as pd
from dash import Dash, html, dcc, Input, Output, State, Patch
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash_ag_grid as dag

df['origin'] = df['origin'].astype(str)
df['cylinders'] = df['cylinders'].astype(str)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

heading = html.H1("Auto MPG Dashboard", className="bg-primary text-white p-3 mb-4 text-center")

# FILTERS
origin_filter = html.Div([
    dbc.Label("Select Origin", className="fw-bold"),
    dcc.Dropdown(
        id='origin-filter',
        options=[{'label': i, 'value': i} for i in ['All'] + sorted(df['origin'].unique())],
        value=['All'],
        multi=True,
        placeholder="Select car origin"
    )
], className="mb-3")

cylinder_filter = html.Div([
    dbc.Label("Select Cylinders", className="fw-bold"),
    dcc.Dropdown(
        id='cylinder-filter',
        options=[{'label': i, 'value': i} for i in ['All'] + sorted(df['cylinders'].unique())],
        value=['All'],
        multi=True,
        placeholder="Select number of cylinders"
    )
], className="mb-3")

control_panel = dbc.Card(
    dbc.CardBody([origin_filter, cylinder_filter]),
    className="bg-light mb-4"
)

info_card = dbc.Card(
    dbc.CardBody([
        dcc.Markdown("""
        ### Dashboard Info
        Explore relationships between car specifications and fuel efficiency.
        - **MPG**: Miles per Gallon (fuel efficiency)
        - **Weight**: Vehicle weight in pounds
        - **Origin**: Manufacturing region (1: USA, 2: Europe, 3: Japan)
        """)
    ]), className="mb-4"
)

# STATISTICS CARDS
stats_cards = dbc.Row([
    dbc.Col(dbc.Card([
        dbc.CardHeader("Average MPG", className="text-center"),
        dbc.CardBody([
            html.H4(id="avg-mpg", className="card-title text-center text-primary")
        ])
    ], color="light"), md=3),

    dbc.Col(dbc.Card([
        dbc.CardHeader("Highest MPG", className="text-center"),
        dbc.CardBody([
            html.H4(id="highest-mpg", className="card-title text-center text-success")
        ])
    ], color="light"), md=3),

    dbc.Col(dbc.Card([
        dbc.CardHeader("Most Common Cylinders", className="text-center"),
        dbc.CardBody([
            html.H4(id="most-cylinders", className="card-title text-center text-warning")
        ])
    ], color="light"), md=3),

    dbc.Col(dbc.Card([
        dbc.CardHeader("Total Cars", className="text-center"),
        dbc.CardBody([
            html.H4(id="total-cars", className="card-title text-center text-info")
        ])
    ], color="light"), md=3)
], className="mb-4")

app.layout = dbc.Container([
    heading,
    stats_cards,
    dbc.Row([
        dbc.Col([control_panel, info_card], md=3),
        dbc.Col([
            # ROW 1:
            dbc.Row([
                dbc.Col([
                    #html.H4("Weight vs MPG Scatter Plot", className="text-center mb-3"),
                    dcc.Graph(
                        id='scatter-plot',
                        config={'displayModeBar': True},
                        style={'height': '300px'}  # Fixed height
                    )
                ], md=6),
                dbc.Col([
                    #html.H4("MPG Distribution by Origin", className="text-center mb-3"),
                    dcc.Graph(
                        id='bar-plot',
                        config={'displayModeBar': True},
                        style={'height': '300px'}  # Fixed height
                    )
                ], md=6)
            ]),
            # ROW 2:
            dbc.Row([
                dbc.Col([
                    #html.H4("Average MPG Trend by Year", className="text-center mb-3 mt-4"),
                    dcc.Graph(
                        id='line-plot',  # GANTI ID KE LINE-PLOT
                        config={'displayModeBar': True},
                        style={'height': '300px'}
                    )
                ], md=12)
            ])
        ], md=9)
    ]),
    dbc.Row([
        dbc.Col(
            html.Div([
                html.H4("Car Data Table", className="mb-3"),
                dag.AgGrid(
                    id='data-grid',
                    rowData=df.to_dict('records'),
                    columnDefs=[{"field": c} for c in df.columns],
                    style={"height": 400, "width": "100%"},
                    defaultColDef={"sortable": True, "filter": True, "resizable": True},
                )
            ], className="mt-4")
        )
    ])
], fluid=True)

# CALLBACK
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-plot', 'figure'),
     Output('line-plot', 'figure'),
     Output('avg-mpg', 'children'),
     Output('highest-mpg', 'children'),
     Output('most-cylinders', 'children'),
     Output('total-cars', 'children'),
     Output('data-grid', 'rowData')],
    [Input('origin-filter', 'value'),
     Input('cylinder-filter', 'value')]
)
def update_dashboard(selected_origin, selected_cyl):
    # Filter data
    dff = df.copy()
    if 'All' not in selected_origin:
        dff = dff[dff['origin'].isin(selected_origin)]
    if 'All' not in selected_cyl:
        dff = dff[dff['cylinders'].isin(selected_cyl)]

    # SCATTER PLOT
    scatter_fig = px.scatter(
        dff,
        x='weight',
        y='mpg',
        color='origin',
        hover_data=['car_name', 'cylinders', 'horsepower'],
        title="Relationship Between Vehicle Weight and MPG",
        labels={
            'weight': 'Vehicle Weight (lbs)',
            'mpg': 'Miles per Gallon (MPG)',
            'origin': 'Origin',
            'horsepower': 'Horsepower',
            'cylinders': 'Cylinders'
        }
    )
    scatter_fig.update_layout(
        #height=500,
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa',
        hovermode='closest',
        font=dict(size=12),
        showlegend=False
    )

    scatter_fig.update_xaxes(title_font=dict(size=14))
    scatter_fig.update_yaxes(title_font=dict(size=14))

    # BAR CHART
    avg_mpg_by_origin = dff.groupby('origin')['mpg'].mean().reset_index()
    avg_mpg_by_origin.columns = ['origin', 'avg_mpg']

    bar_fig = px.bar(
        avg_mpg_by_origin,
        x='origin',
        y='avg_mpg',
        color='origin',
        title="Average MPG by Origin",
        labels={
            'origin': 'Origin',
            'avg_mpg': 'Miles per Gallon (MPG)'
        },
    )
    bar_fig.update_layout(
        #height=500,
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa',
        font=dict(size=12),
        showlegend=False
    )
    bar_fig.update_xaxes(title_font=dict(size=14))
    bar_fig.update_yaxes(title_font=dict(size=14))

    # LINE CHART
    yearly_avg = dff.groupby('model_year')['mpg'].mean().reset_index()
    yearly_avg.columns = ['model_year', 'avg_mpg']

    line_fig = px.line(
        yearly_avg,
        x='model_year',
        y='avg_mpg',
        title="Average MPG Trend by Model Year",
        labels={
            'model_year': 'Model Year',
            'avg_mpg': 'Average MPG'
        },
        markers=True  # Tambahkan marker di setiap titik
    )
    line_fig.update_layout(
        #height=500,
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa',
        font=dict(size=12),
        showlegend=False
    )
    bar_data = dff['cylinders'].value_counts().reset_index()
    bar_data.columns = ['cylinders', 'count']

    # STATISTICS
    avg_mpg = f"{dff['mpg'].mean():.1f}" if not dff.empty else "N/A"
    highest_mpg = f"{dff['mpg'].max():.1f}" if not dff.empty else "N/A"
    most_cylinders = f"{dff['cylinders'].mode().iloc[0] if not dff.empty else 'N/A'}"
    total_cars = f"{len(dff)}"

    return scatter_fig, bar_fig, line_fig, avg_mpg, highest_mpg, most_cylinders, total_cars, dff.to_dict('records')

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>